In [137]:
my_data = [[ 'hot','high','false','no','sunny'],
        ['hot','high','true','no','sunny'],
        ['hot','high','false','yes','overcast'],
        ['mild','high','false','yes','rainy'],
        ['cool','normal','false','yes','rainy'],
        ['cool','normal','true','no','rainy'],
        ['cool','normal','true','yes','overcast'],
        ['mild', 'high','false','no','sunny'],
        ['cool','normal','false','yes','sunny'],
        ['mild','normal','false','yes','rainy'],
        ['mild','normal','true','yes','sunny'],
        ['mild','high','true','yes','overcast'],
        ['hot', 'normal','false','yes','overcast'],
        ['mild', 'high','true','no','rainy']]


In [122]:
class node:
    def __init__(self,col=-1,value=None,results=None,tb=None,fb=None):
        self.col=col 
        self.value=value 
        self.results=results 
        self.tb=tb # 
        self.fb=fb #

In [123]:

def divideset(rows,column,value):
    
    split_function=None
    # 
    if isinstance(value,int) or isinstance(value,float):
        split_function=lambda row:row[column]>=value
    
    else:
        split_function=lambda row:row[column]==value
   
   # 
    set1=[row for row in rows if split_function(row)] 
    set2=[row for row in rows if not split_function(row)]
    return (set1,set2)

([['hot', 'high', 'false', 'no', 'sunny'],
  ['hot', 'high', 'true', 'no', 'sunny'],
  ['hot', 'high', 'false', 'yes', 'overcast'],
  ['mild', 'high', 'false', 'yes', 'rainy'],
  ['mild', 'high', 'false', 'no', 'sunny'],
  ['mild', 'high', 'true', 'yes', 'overcast'],
  ['mild', 'high', 'true', 'no', 'rainy']],
 [['cool', 'normal', 'false', 'yes', 'rainy'],
  ['cool', 'normal', 'true', 'no', 'rainy'],
  ['cool', 'normal', 'true', 'yes', 'overcast'],
  ['cool', 'normal', 'false', 'yes', 'sunny'],
  ['mild', 'normal', 'false', 'yes', 'rainy'],
  ['mild', 'normal', 'true', 'yes', 'sunny'],
  ['hot', 'normal', 'false', 'yes', 'overcast']])

In [125]:
def uniquecounts(rows):
    results={}
    for row in rows:
        
        r=row[len(row)-1]
        if r not in results: results[r]=0
        results[r]+=1
    return results

In [25]:
from collections import defaultdict
def uniquecounts_dd(rows):
    results = defaultdict(lambda: 0)
    for row in rows:
        r = row[len(row)-1]
        results[r]+=1
    return dict(results) 

In [126]:
uniquecounts(my_data), uniquecounts_dd(my_data)

({'overcast': 4, 'rainy': 5, 'sunny': 5},
 {'overcast': 4, 'rainy': 5, 'sunny': 5})

In [127]:
def entropy(rows):
    from math import log
    log2=lambda x:log(x)/log(2)  
    results=uniquecounts(rows)
    ent=0.0
    for r in results.keys():
        p=float(results[r])/len(rows) 
        ent=ent-p*log2(p)
    
    return ent

In [128]:
entropy(my_data)

1.5774062828523452

In [129]:
entropy([my_data[i][0:1] for i in range(0,14)])

1.5566567074628228

In [130]:
entropy([my_data[i][1:2] for i in range(0,14)])

1.0

In [131]:
entropy([my_data[i][2:3] for i in range(0,14)])

0.9852281360342516

In [132]:
entropy([my_data[i][3:4] for i in range(0,14)])

0.9402859586706309

In [133]:
def tree(rows, scorefun=entropy):
    if len(rows) == 0: return node()
    current_score = scorefun(rows)

    best_gain = 0.0
    best_criteria = None
    best_sets = None

    column_count = len(rows[0]) - 1	
    for col in range(0, column_count):
        column_values = set([row[col] for row in rows])

        for value in column_values:
            set1, set2 = divideset(rows, col, value)

            p = float(len(set1)) / len(rows)
            gain = current_score - p*scorefun(set1) - (1-p)*scorefun(set2)
            if gain > best_gain and len(set1) > 0 and len(set2) > 0:
                best_gain = gain
                best_criteria = (col, value)
                best_sets = (set1, set2)

    if best_gain > 0:
        trueBranch = tree(best_sets[0])
        falseBranch = tree(best_sets[1])
        return node(col=best_criteria[0], value=best_criteria[1],
                tb=trueBranch, fb=falseBranch)
    else:
        return node(results=uniquecounts(rows))

In [138]:
def tree_print(tree,indent='    '):
    #  leaf
    if tree.results!=None:
        print str(tree.results)
    else:
        print 'Column ' + str(tree.col)+' : '+str(tree.value)+' '

        # ranches
        print indent+'True->',
        tree_print(tree.tb,indent+'  ')
        print indent+'False->',
        tree_print(tree.fb,indent+'  ')

In [140]:
my_data[0:5]

[['hot', 'high', 'false', 'no', 'sunny'],
 ['hot', 'high', 'true', 'no', 'sunny'],
 ['hot', 'high', 'false', 'yes', 'overcast'],
 ['mild', 'high', 'false', 'yes', 'rainy'],
 ['cool', 'normal', 'false', 'yes', 'rainy']]

In [139]:
tree_print(tree(my_data))

Column 3 : no 
    True-> Column 0 : hot 
      True-> {'sunny': 2}
      False-> Column 2 : false 
        True-> {'sunny': 1}
        False-> {'rainy': 2}
    False-> Column 0 : hot 
      True-> {'overcast': 2}
      False-> Column 2 : true 
        True-> Column 0 : mild 
          True-> Column 1 : high 
            True-> {'overcast': 1}
            False-> {'sunny': 1}
          False-> {'overcast': 1}
        False-> Column 0 : mild 
          True-> {'rainy': 2}
          False-> {'rainy': 1, 'sunny': 1}
